<a href="https://colab.research.google.com/github/VinayRawat2004/IMBD-NLP-/blob/main/imbdNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

THE FIRST STEP IS CHANGING THE RUNTIME TYPE FROM CPU TO GPU
T4 GPU AS I WILL USE NEURAL NETWORK ALGO (LSTM)

In [1]:
!pip install kaggle

DEPENDENCIES IMPORT, USING KAGGLE LIBRARY BECAUSE OF THE DATSET I USED

In [2]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



*   DATA COLLECTION
*   FROM KAGGLE API : json.laod converts json object into a python dictionary



In [4]:
kag_api = json.load(open("kaggle.json"))

set kaggle credentials as environmental variable, dont directly insert the key as if you share your file with someone

In [5]:
os.environ["KAGGLE_USERNAME"] = kag_api["username"]
os.environ["KAGGLE_KEY"] = kag_api["key"]

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 41.8MB/s]
100% 25.7M/25.7M [00:00<00:00, 130MB/s] 


In [7]:
!ls


imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


UNZIP file

In [8]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip:
  zip.extractall()

In [9]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


LOAD DATA

In [10]:
data = pd.read_csv("IMDB Dataset.csv")

In [11]:
data.shape

(50000, 2)

In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


### **EDA**

In [14]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [15]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-15-329ec005e21e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [16]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [18]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


SPLIT THE DATA INTO TEST AND TARAIN DATA

In [19]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [20]:
train_data.shape

(40000, 2)

In [21]:
test_data.shape

(10000, 2)

**DATA PREPROCESS STEPS**


1.   TOKENIZE TEXT DATA
2.   List item



In [22]:
tok= Tokenizer(num_words=5000)
tok.fit_on_texts(train_data['review'])
X_train = pad_sequences(tok.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tok.texts_to_sequences(test_data["review"]),maxlen=200)

In [23]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [24]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


IT DOESNT HAVE THE SENTIMENT OR OUTPUT, LET'S MAKE TWO MORE

In [25]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [26]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


In [27]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


# LSTM

In [28]:
model = Sequential()  #stack layers
model.add(Embedding(input_dim=5000, output_dim=128,input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
model.build(input_shape=(None, 200))
print(model.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

TRAIN THE MODEL

In [32]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 248ms/step - accuracy: 0.8416 - loss: 0.3760 - val_accuracy: 0.8556 - val_loss: 0.3469
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 248ms/step - accuracy: 0.8725 - loss: 0.3110 - val_accuracy: 0.8581 - val_loss: 0.3503
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 144s 252ms/step - accuracy: 0.8898 - loss: 0.2783 - val_accuracy: 0.8706 - val_loss: 0.3189
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 125s 249ms/step - accuracy: 0.8983 - loss: 0.2559 - val_accuracy: 0.8690 - val_loss: 0.3520
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 248ms/step - accuracy: 0.9061 - loss: 0.2334 - val_accuracy: 0.8761 - val_loss: 0.3068


MODEL EVALUATION

In [35]:
loss, accuracy=model.evaluate(X_test, Y_test)
print("TEST LOSS:",loss*100)
print("TEST ACCURACY:",accuracy*100)

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - accuracy: 0.8802 - loss: 0.2893
TEST LOSS: 28.704065084457397
TEST ACCURACY: 88.37000131607056


PREDICTION SYSTEM

In [36]:
def predict_sentiment(review):
  # tokenize and then pad the review
  sequence = tok.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

EXAMPLES

In [37]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
The sentiment of the review is: positive


In [38]:
new_review = "This movie was not so good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
The sentiment of the review is: negative


In [39]:
new_review = "Did not like the characters, movie was not so good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
The sentiment of the review is: negative


In [40]:
new_review = "this was hands down the best movie I’ve ever seen in my life. But, there were a couple of moments where the pacing dragged a bit. Still, it didn’t take away from how amazing it was overall"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
The sentiment of the review is: positive


In [41]:
new_review = "I really wanted to like this movie, but it felt like a missed opportunity. The performances were top-notch, but the story was flat, and the direction was off. It’s just not what I expected."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
The sentiment of the review is: negative


In [44]:
new_review = "movie was genuinely so good and exciting,the charcters though were average"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
The sentiment of the review is: positive


In [45]:
model.save('IMBdModel.h5')  # Save the model in HDF5 format

# To download the model to your local machine
from google.colab import files
files.download('IMBdModel.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
import joblib
from google.colab import files
joblib.dump(tok, "tokenizer.pkl")

files.download('tokenizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>